In [1]:
import pandas as pd
import numpy as np
import scipy
import sklearn

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel, SelectPercentile, chi2, VarianceThreshold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import metrics

In [4]:
import optuna
from optuna.visualization import plot_intermediate_values

In [5]:
import numpy as np
import xgboost as xgb
import lightgbm as lgb

In [7]:
SEED = 85

In [8]:
from XGBoostOptuna import *
from optuna_utils import *

# Get Dataset

In [9]:
from scipy.sparse import csr_matrix, save_npz, load_npz

In [10]:
X_train = load_npz("X_train.npz")
y_train = pd.read_pickle("y_train.pkl")

In [11]:
X_test = load_npz("X_test.npz")
y_test = pd.read_pickle("y_test.pkl")

In [12]:
#pipeline = Pipeline([("sel", SelectPercentile(chi2))])
#pipe_params = {"sel__percentile" : optuna.distributions.IntUniformDistribution(1,100)}
pipeline = None
pipe_params = None

In [13]:
#TODO: automatic seed
xgb_linear = XGBClassifierOptuna(booster_list=['gblinear'])
xgb_tree = XGBClassifierOptuna(booster_list=['gbtree', 'dart'])
#lgb_estimator = LGBClassifierOptuna(seed=SEED)
#sgd_estimator = SGDClassifierOptuna(seed=SEED)
#rf_estimator  = RFClassifierOptuna(seed=SEED)

In [14]:
estimators = [xgb_linear]#, xgb_tree]#, sgd_estimator]

In [15]:
studies = Study(X_train, y_train, estimators, pipeline, pipe_params, 
              cv=StratifiedKFold(n_splits=3, random_state=SEED), max_iter=200, max_fails=3,
              scoring=sklearn.metrics.roc_auc_score, greater_is_better=True, random_state=SEED)

In [16]:
%%time
studies.run(n_trials_per_estimator=15, n_jobs=-1)

XGBoost Classifier
Score: 0.5
Score: 0.5
Score: 0.5
Score: 0.96409
Wall time: 22.8 s


In [17]:
study = studies.studies[0]

In [18]:
study.best_value

0.964092541642688

In [19]:
plot_intermediate_values(study)

In [20]:
final_pipeline = study.best_trial.user_attrs['pipeline']

In [21]:
study.best_trial.value

0.964092541642688

In [22]:
final_pipeline = final_pipeline.fit(X_train, y_train)

In [23]:
test_probs = final_pipeline.predict_proba(X_test)
test_probs = test_probs[:,1]

In [24]:
sklearn.metrics.roc_auc_score(y_test, test_probs)

0.9706423959090476